In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import pyarrow.parquet as pq
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
file_path = 'C:/Users/DELL/Desktop/Untitled Folder/yellow_tripdata_2022-01.parquet'
table = pq.read_table(file_path)

In [ ]:
df = table.to_pandas()


# Check for errors, duplicates, null

## Remove NA

In [ ]:
df.isna().sum()

In [ ]:
df1 = df.dropna()

## Remove Errors (<0 values)

In [ ]:
columns_to_check = ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee']

In [ ]:
df2 = df1[~(df1[columns_to_check] < 0).any(axis=1)]

## Remove Duplicates

In [ ]:
df2.duplicated().sum()

## Remove Invalid data

In [ ]:
df2.columns

In [ ]:
df2['VendorID'].unique()

In [ ]:
df2 = df2[df['tpep_pickup_datetime'] < df['tpep_dropoff_datetime']]

In [ ]:
df2['passenger_count'].unique()

In [ ]:
df2 = df2[df2['passenger_count'] > 0]

In [ ]:
df2 = df2[df2['trip_distance'] > 0]

In [ ]:
df2['RatecodeID'].unique()

In [ ]:
df2 = df2[df2['RatecodeID'] != 99]

In [ ]:
df2['store_and_fwd_flag'].unique()

In [ ]:
df2['PULocationID'].unique()

In [ ]:
df2['DOLocationID'].unique()

In [ ]:
df2['payment_type'].unique()

In [ ]:
df2 = df2[df2['fare_amount'] > 0]

In [ ]:
df2 = df2[df2['total_amount'] > 0]

In [ ]:
df2 = df2[df2['fare_amount'] != 401092.32]

In [ ]:
df2.reset_index(drop=True, inplace=True)

## FEATURE UNDERSTANDING

In [ ]:
df2.describe()

In [ ]:
df2.info()

In [ ]:
df2.head(10)

# Data Transformation

In [ ]:
df2['duration'] =  df2['tpep_dropoff_datetime'] - df2['tpep_pickup_datetime']

In [ ]:
df2['total_charges'] = df2['fare_amount'] + df2['extra'] + df2['mta_tax'] + df2['tolls_amount'] + df2['improvement_surcharge'] + df2['tip_amount'] + df2['congestion_surcharge'] + df2['airport_fee'] 

In [ ]:
df2 = df2.drop(columns ='total_amount')

In [ ]:
df2

In [ ]:
df2['PULocationID'].value_counts().sort_values(ascending = False)

In [ ]:
df2['DOLocationID'].value_counts().sort_values(ascending = False)

## CREATE LOOKUP TABLES AND FACT TABLE

In [ ]:
df2['trip_ID'] = df2.index

In [ ]:
Vendor_name = {
    1:"Creative Mobile Technologies, LLC",
    2:"VeriFone Inc.",
}


Vendor_lookup = pd.DataFrame({'VendorID': df2['VendorID'].unique()})
Vendor_lookup['Vendor_name'] = Vendor_lookup['VendorID'].map(Vendor_name)

Vendor_lookup


In [ ]:
PU_datetime_lookup = pd.DataFrame({'PU_datetime': df2['tpep_pickup_datetime'].unique()})
PU_datetime_lookup['PU_hour'] = PU_datetime_lookup['PU_datetime'].dt.hour
PU_datetime_lookup['PU_day'] = PU_datetime_lookup['PU_datetime'].dt.day
PU_datetime_lookup['PU_month'] = PU_datetime_lookup['PU_datetime'].dt.month
PU_datetime_lookup['PU_year'] = PU_datetime_lookup['PU_datetime'].dt.year
PU_datetime_lookup['PU_weekday'] = PU_datetime_lookup['PU_datetime'].dt.weekday

In [ ]:
PU_datetime_lookup

In [ ]:
DO_datetime_lookup = pd.DataFrame({'DO_datetime': df2['tpep_dropoff_datetime'].unique()})

In [ ]:
DO_datetime_lookup['DO_hour'] = DO_datetime_lookup['DO_datetime'].dt.hour
DO_datetime_lookup['DO_day'] = DO_datetime_lookup['DO_datetime'].dt.day
DO_datetime_lookup['DO_month'] = DO_datetime_lookup['DO_datetime'].dt.month
DO_datetime_lookup['DO_year'] = DO_datetime_lookup['DO_datetime'].dt.year
DO_datetime_lookup['DO_weekday'] = DO_datetime_lookup['DO_datetime'].dt.weekday

In [ ]:
DO_datetime_lookup

In [ ]:
RateCode_name = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}


RateCode_lookup = pd.DataFrame({'RatecodeID': df2['RatecodeID'].unique()})
RateCode_lookup['RateCode_name'] = RateCode_lookup['RatecodeID'].map(RateCode_name )

RateCode_lookup 

In [ ]:
RateCode_lookup = RateCode_lookup.sort_values(by='RatecodeID', ascending=True)

RateCode_lookup.reset_index(drop=True, inplace=True )

RateCode_lookup

In [ ]:
Payment_type = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

Payment_lookup = pd.DataFrame({'Payment_typeID': df2['payment_type'].unique()})
Payment_lookup['Payment_type'] = Payment_lookup['Payment_typeID'].map(Payment_type)

Payment_lookup = Payment_lookup.sort_values(by='Payment_typeID', ascending=True)

Payment_lookup.reset_index(drop=True, inplace=True )

Payment_lookup

# EXPORT TO CSV

In [ ]:
## df2.to_csv("fact_table.csv")
## Vendor_lookup.to_csv("Vendor_lookup.csv")
##PU_datetime_lookup.to_csv("PU_datetime_lookup.csv")
##DO_datetime_lookup.to_csv("DO_datetime_lookup.csv")
##RateCode_lookup.to_csv("RateCode_lookup.csv")
##Payment_lookup.to_csv("Payment_lookup.csv")